In [ ]:
'''
 # @ Author: [Author name removed]
 # @ Create date: 2022-01-13
 # @ Modified by: [Name removed]
 # @ Modified date: 2024-11-15
 # @ Description: This file implements functions from the pricing module to show how to get the price and yield of a trade.
 '''
# ensures that any changes to the modules will be reloaded when this cell is run
%load_ext autoreload
%autoreload 2

import os
import pickle
import numpy as np

from google.cloud import bigquery


# importing from parent directory: https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
sys.path.insert(0,'../../')

from ficc.utils.auxiliary_functions import sqltodf
from ficc.utils.process_features import process_features
from ficc.pricing.price import compute_price
from ficc.pricing.yield_rate import compute_yield

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '~/creds.json'    # put the location of your `creds.json` file

### Conditions that we tested on:
* `yield > 0`: conceptually, this notebook should work for any yield, but in practice, bonds with zero yield indicates a problem with MSRB data, and negative yields haven't been tested on
* `is_non_transaction_based_compensation is false`, `is_lop_or_takedown is false`: conceptually, this notebook should work without these filters, but in practice, away from market prices sometimes have anomolous yields reported in the MSRB data
* `callable_at_cav is false`: this notebook will not be correct for bonds callable at cav; future work will incorporate the special case of cav bonds that have call prices from the reference data
* `ref_valid_to_date > timestamp(publish_datetime, "America/New_York")`, `timestamp(publish_datetime, "America/New_York") >= ref_valid_from_date`, `msrb_valid_to_date > publish_datetime`, `msrb_valid_from_date <= publish_datetime`: conditions to correctly join the reference data and MSRB
* `par_call_price = 100`: conceptually this is identical to the fourth bullet point, but sometimes there are issues in the reference data
* `interest_payment_frequency is not NULL`: if interest payment frequency is NULL, then the coupon frequency is unknown; possibly the field `coupon_type` could be used to determine the interest payment frequency

In [ ]:
query = lambda trade_date: ''' SELECT
                                IFNULL(settlement_date, assumed_settlement_date) AS settlement_date,
                                trade_date,
                                cusip,
                                par_traded,
                                accrual_date,
                                dollar_price,
                                issue_price,
                                coupon,
                                interest_payment_frequency,
                                next_call_date,
                                par_call_date,
                                next_call_price,
                                par_call_price,
                                maturity_date,
                                previous_coupon_payment_date,
                                next_coupon_payment_date,
                                first_coupon_date,
                                coupon_type,
                                muni_security_type,
                                called_redemption_type,
                                refund_date,
                                refund_price,
                                is_callable,
                                is_called,
                                call_timing,
                                yield,
                                rtrs_control_number,
                                has_zero_coupons,
                                last_period_accrues_from_date, 
                                call_defeased, 
                                issue_amount,    -- needed for `process_features(...)`
                                maturity_amount,    -- needed for `process_features(...)`
                                orig_principal_amount,    -- needed for `process_features(...)`
                                max_amount_outstanding,    -- needed for `process_features(...)`
                                delivery_date,    -- needed for `process_features(...)`
                                next_sink_date    -- needed for `process_features(...)`
                            FROM 
                                `eng-reactor-287421.auxiliary_views.trades_with_ref_data_pd`
                            WHERE 
                                -- yield > 0 and
                                -- is_non_transaction_based_compensation is false and 
                                -- callable_at_cav is false and 
                                -- is_lop_or_takedown is false and 
                                ref_valid_to_date > timestamp(publish_datetime, "America/New_York") and
                                timestamp(publish_datetime, "America/New_York") >= ref_valid_from_date and
                                msrb_valid_to_date > publish_datetime and
                                msrb_valid_from_date <= publish_datetime and
                                -- par_call_price is not NULL and
                                -- par_call_price = 100 and
                                -- interest_payment_frequency is not NULL and 
                                -- interest_payment_frequency = 16 and 
                                -- coupon > 0 and
                                trade_date = \'''' + trade_date + '''\'
                            '''

In [ ]:
date = '2024-07-31'

In [ ]:
muni_df_query = query(date)

using_saved_muni_df = False
MUNI_DF_FILE_NAME = 'muni_df_and_query.pkl'
if os.path.exists(MUNI_DF_FILE_NAME):
    with open(MUNI_DF_FILE_NAME, 'rb') as file:
        muni_df_query_from_pkl, muni_df = pickle.load(file)
    if muni_df_query == muni_df_query_from_pkl:
        using_saved_muni_df = True

if not using_saved_muni_df:
    bqclient = bigquery.Client()
    muni_df = sqltodf(muni_df_query, bqclient)
    with open(MUNI_DF_FILE_NAME, 'wb') as file:
        pickle.dump((muni_df_query, muni_df), file)

In [ ]:
muni_df    # visualize the dataframe

In [ ]:
sorted(muni_df.columns)

In [ ]:
muni_df = process_features(muni_df)

In [ ]:
muni_df = muni_df.rename(columns={'yield': 'ficc_ytw'})

Apply the `compute_price` function to every trade.

In [ ]:
muni_df['price_calc_from_yield'] = muni_df.apply(lambda x: compute_price(x), axis=1)

Plot the price from the reference data on the x-axis and our computed price on the y-axis.

In [ ]:
muni_df['price_from_yield'] = [x[0] for x in muni_df['price_calc_from_yield']]
muni_df.plot.scatter(x='dollar_price', y='price_from_yield', c='DarkBlue')

Return the sum and mean of the errors where the error is defined as the absolute distance between our computed price and the reference price.

In [ ]:
muni_df['price_delta'] = abs(muni_df['price_from_yield'] - muni_df['dollar_price'])
print(f'Sum of errors: {np.sum(muni_df['price_delta'])}')
print(f'Mean of errors: {np.mean(muni_df['price_delta'])}')

Apply the `compute_yield` function to every trade. The below code may raise a `RuntimeError` since `compute_yield(...)` may not converge since it uses Newton's method.

In [ ]:
muni_df['yield_and_calc_date'] = muni_df.apply(lambda x: compute_yield(x), axis=1)

Plot the yield from the reference data on the x-axis and our computed yield on the y-axis.

In [ ]:
muni_df['yield_from_price'] = [x[0] for x in muni_df['yield_and_calc_date']]
muni_df['yield_calc_date'] = [x[1] for x in muni_df['yield_and_calc_date']]
# when we get close to redemption, it's difficult to compute the yield from the price
muni_df.plot.scatter(x='yield', y='yield_from_price', c='DarkBlue')

Return the sum and mean of the errors where the error is defined as the absolute distance between our computed yield and the reference yield.

In [ ]:
muni_df['ytw_delta'] = abs(muni_df['yield_from_price'] - muni_df['yield'])
print(f'Sum of errors: {np.sum(muni_df['ytw_delta'])}')
print(f'Mean of errors: {np.mean(muni_df['ytw_delta'])}')